##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [8]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

      **评分标准**：
      - 强烈负面 (-5): 极端消极
      - 负面 (-3 ~ -4): 较为消极
      - 轻微负面 (-1 ~ -2): 略微消极
      - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
      - 轻微正面 (+1 ~ +2): 略微积极
      - 正面 (+3 ~ +4): 较为积极
      - 强烈正面 (+5): 极端积极

      **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子**。
      **新闻文章内容:**
      {sentences}

      **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [9]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [12]:
import json

# JSON 파일 로드
file_path = "../data/sample_4.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 4개의 기사 로드 완료


## 감성분석 수행 함수

In [10]:
# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 기사 본문만 전달
        input_data = {"sentences": article["sentences"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [13]:
output_path = "../data/0307/results_smaple4_문장별입력_문장별분석.json"
run_sentiment_analysis(data, output_path)

2025-03-07 20:17:52,478 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['新华社平壤６月２９日电记者郭一娜 陆睿据朝鲜劳动党机关报劳动新闻２９日报道朝鲜外务省裁军与和平研究所发言人谴责美日韩导弹防御联合军演称该军演是对朝鲜的又一重大军事挑衅破坏地区和平与安全。', '该发言人是在答朝中社记者提问时做出上述表述的。', '发言人说２８日美国以应对朝鲜核导威胁为由联合日韩在夏威夷附近海域进行导弹防御联合军演该军演是美日韩公然举行带有挑衅性的三方联合军演。', '发言人指出以美国在亚太地区的导弹防御系统和美日韩三角军事同盟为基础的亚洲版北约的出现将招致东北亚地区在不久后展开新一轮军备竞赛和新冷战。', '而这正是此次军演的危险性所在。', '发言人表示以美国为首的敌对势力频繁进行威胁朝鲜和地区安全的军演。', '在此情况下朝鲜军民将扩大并强化能够给予敌人威胁的先发制人核打击能力继续研发多样化战略性进攻武器。'], 'text': '-3\n0\n-3\n-3\n-2\n-2\n-1'}
 '朝鲜谴责美日韩联合军演' 분석 완료.


2025-03-07 20:17:53,485 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['资料图萨德反导系统 人民网北京12月13日电 邱越据韩联社11日报道韩国国防部当天透露正在以最快速度处理萨德部署事宜并制定了明年5月部署完毕的方针。', '军事专家尹卓在接受央视采访时表示韩美军方为了防止韩国接任政府对萨德部署产生颠覆性影响所以希望能够抢在韩国政权更迭前完成萨德的部署。', '据报道韩美7月宣布在韩国布署萨德的决定时给部署完成设定的时间是2017年年底前。', '有分析认为针对总统朴槿惠的弹劾案在国会通过最快可能在明年春天提前举行总统大选在此大背景下韩军加快部署萨德实际上是要将总统大选对部署萨德的影响最小化。', '目前韩国在野党仍强烈反对部署萨德韩军强推萨德在野党与军方的矛盾或将空前尖锐。', '韩联社11日援引国防部相关人士的话说作为预定部署地点的星州高尔夫球场电力下水道路等基础设施完备不需要新建多少设施。', '韩国军方认为如果一切顺利部署萨德的所有程序可以在6个月内完成最快明年5月即可完成部署。', '本月4日驻韩美军司令也曾表达希望提前完成萨德部署的意愿称萨德可以在未来8至10个月完成部署。', '据俄媒报道此前有记者向美国国防部发言人彼得库克提问称韩国总统朴槿惠辞职或被弹劾会否改变在韩部署萨德的计划？', '库克表示五角大楼不会改变在韩国部署萨德反导系统的计划。', '尹卓认为库克的话实际是在向韩国军方和国防部提出要求在韩部署萨德牵涉到美国在亚太的安全利益要防止下任领导人上台后使在韩部署萨德的成果作废。', '尹卓指出韩国军方与美国军方有共同利益而他们与未来可能接任朴槿惠的大部分候选人在部署萨德问题上有着重大分歧为了防止接任政府对萨德部署产生颠覆性的影响所以美国希望能抢在韩国政权更迭之前完成萨德的部署。', '此外萨德系统的部署可使美日韩形成准军事同盟关系即便韩国新任领导上台也很难将其废除。', '军事专家李莉表示如今韩国加快部署萨德的态势很明显几乎可以算进入了倒计时阶段。', '即便明年1月特朗普就任美国总统加快在韩部署萨德的意愿也不会有所改变。', '而且韩国的确有可能在明年5月份让萨德系统发射车导弹预警装置和电源车全部就位星洲高尔夫球场。'], 'text': '-1\n-2\n-1\n-2\n-3\n0\n0\n0\n-1\n0\n-1\n-2\n-1\n-1\n0

2025-03-07 20:17:54,015 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['韩日军事协定可能是未来或下一届韩国政府的一个负担甚至是苦果。', '11月23日韩日正式签署军事情报保护协定GSOMIA。', '尽管该协定规定仅可共享二级三级等非核心的军事秘密如应对朝鲜的核武器以及导弹的威胁等有关的情报但此协定仍然标志着韩日两国在针对朝鲜导弹预警情报的分享方面绕开美国迈进了实质性的一步此外也是颇有长远影响的一点即美日韩三边军事关系有可能变得更为布局合理从而为长期稳固化创造一定的条件。', '那么在复杂多变的东北亚地区局势中此举会对韩国带来什么影响？', '笔者认为首先就目前的韩国政治乱局而言该协定的签署并未取得民众的充分理解也没有获得在野党的广泛共识而如此果断地签署该协定颇有表明政治决心的意味。', '即表明不论时局如何变化本届政府内预定的方案始终是要坚决地推行的。', '那么以此推敲韩日军事协定可能是未来或下一届韩国政府的一个负担甚至是苦果。', '而未来青瓦台会面临至少两方面的压力第一国内民众的反对。', '据盖洛普韩国最新统计对于该协定来自执政党新国家党方面的支持率仅有55而在野党方面的反对率则高达70。', '总体而言仅有30左右的受访人认为该协定对韩国安全有帮助。', '因此在目前的政治气氛中该协定的签署更像是政府的政策而非国家的国策。', '第二就该协定的实质而言其实效性也可能会受到质疑。', '首当其冲的就是信任问题。', '朝鲜半岛本身地域狭小这就决定了导弹预警对韩国来说其实质效应可能会小于对于日本的实质效应因此表面看来这是一个共享军事情报的协定但实质上日本可能更期望能利用韩国的早期预警情报系统来有效应对来自朝鲜的导弹威胁。', '这就有可能埋下一个有关可操作性甚至是互信的潜在问题。', '那么从地区的角度看该协定是不是就表明美日韩三边关系至少在军事同盟层面上是否就向铁三角结构转变了呢？', '长久以来美日韩三边同盟关系是一个不稳定的三角形结构即两条实线美日同盟美韩同盟和一条虚线韩日关系。', '朴槿惠政府看似是解决了韩日间至少两个长期以来悬而未决的问题慰安妇问题和军事合作关系问题但不确定性仍然存在。', '首先从韩国国内政治看以上两个协议的签订并非建立在牢靠的民意及党派意见基础之上反映的并不一定是此刻韩国的举国共识其次外部的变数依旧存在比如美日韩三边关系始终不可避

2025-03-07 20:17:55,101 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['央视网消息美日韩三国同盟一直都是美国在推动而韩日两国之间的历史问题根深蒂固韩国政坛动荡韩日军事情报合作进行地也不顺利在美国方面特朗普政府即将就职但目前新政府的地区战略并不明朗诸多政策还有不确定性缺乏美国力量的推动美日韩三国同盟关系将缺乏动力。'], 'text': '-2\n-3\n-2\n-2\n-3'}
 '慰安妇少女像引爆韩日矛盾:美日韩同盟缺乏动力' 분석 완료.
 전체 기사 감성 분석 완료. ../data/0307/results_smaple4_문장별입력_문장별분석.json
